In [1]:
!unzip 'nlp assignment 3.zip'

Archive:  nlp assignment 3.zip
   creating: nlp assignment 3/
  inflating: nlp assignment 3/test.txt  
  inflating: nlp assignment 3/train.txt  


In [2]:
cd 'nlp assignment 3'

/content/nlp assignment 3


In [0]:
f = open("train.txt","r")
uncleaned = f.readlines()
train_data=[]

i=0
while(i<len(uncleaned)):
  x=uncleaned[i]
  l = x.split()
  if(l!=[]):
    if(l[0]=='meta'):
      d=dict()
      d['id']=int(l[1])
      d['sentiment']=str(l[2])
      d['tokens']=[]
      d['lang_id']=[]
    elif(len(l)==2):
      d['tokens'].append(l[0].lower())
      d['lang_id'].append(l[1])
  else:
    train_data.append(d)
  i=i+1

train_labels = []
for i in range(len(train_data)):
  s = train_data[i]['sentiment']
  train_labels.append(s)

In [0]:
f = open("test.txt","r")
uncleaned = f.readlines()
test_data=[]

i=0
while(i<len(uncleaned)):
  x=uncleaned[i]
  l = x.split()
  if(l!=[]):
    if(l[0]=='meta'):
      d=dict()
      d['id']=int(l[1])
      d['sentiment']=str(l[2])
      d['tokens']=[]
      d['lang_id']=[]
    elif(len(l)==2):
      d['tokens'].append(l[0].lower())
      d['lang_id'].append(l[1])
  else:
    test_data.append(d)
  i=i+1

test_labels = []
for i in range(len(test_data)):
  s = test_data[i]['sentiment']
  test_labels.append(s)

In [35]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
max_chars = 25
vector_size = 128

In [0]:
train_data_1 = []
for i in range(len(train_data)):
  t = train_data[i]['tokens']
  l = train_data[i]['lang_id']
  t_filter = [t[x] for x in range(len(t)) if l[x]!='O']
  t_trigram = [' '.join(t_filter[i:i+3]) for i in range(len(t_filter)-2)]
  #t_trigram += [' '.join(t_filter[i:i+4]) for i in range(len(t_filter)-3)]
  #t_trigram += [' '.join(t_filter[i:i+5]) for i in range(len(t_filter)-4)]
  #t_trigram += [' '.join(t_filter[i:i+6]) for i in range(len(t_filter)-5)]
  train_data_1.append(t_trigram)

In [0]:
test_data_1 = []
for i in range(len(test_data)):
  t = test_data[i]['tokens']
  l = test_data[i]['lang_id']
  t_filter = [t[x] for x in range(len(t)) if l[x]!='O']
  t_trigram = [' '.join(t_filter[i:i+3]) for i in range(len(t_filter)-2)]
  #t_trigram += [' '.join(t_filter[i:i+4]) for i in range(len(t_filter)-3)]
  #t_trigram += [' '.join(t_filter[i:i+5]) for i in range(len(t_filter)-4)]
  #t_trigram += [' '.join(t_filter[i:i+6]) for i in range(len(t_filter)-5)]
  test_data_1.append(t_trigram)

In [0]:
train_labels_1 = []
for i in range(len(train_labels)):
  s = train_labels[i]
  if(s=='negative'):
    train_labels_1.append([1, 0, 0])
  elif(s=='positive'):
    train_labels_1.append([0, 0, 1])
  else:
    train_labels_1.append([0, 1, 0])

In [0]:
test_labels_1 = []
for i in range(len(test_labels)):
  s = test_labels[i]
  if(s=='negative'):
    test_labels_1.append([1, 0, 0])
  elif(s=='positive'):
    test_labels_1.append([0, 0, 1])
  else:
    test_labels_1.append([0, 1, 0])

In [0]:
import gensim
import numpy as np

In [0]:
y_train_1 = np.array(train_labels_1)
#y_test_1 = np.array(test_labels_1)

In [0]:
word_model_1 = gensim.models.Word2Vec(train_data_1+test_data_1, size=vector_size, seed=1011, min_count=1, iter=100)

In [0]:
word_vectors = word_model_1.wv

In [0]:
X_train = np.zeros((len(train_data_1), max_chars, vector_size))
X_test = np.zeros((len(test_data_1), max_chars, vector_size))

In [0]:
for i in range(len(train_data_1)):
  t = train_data_1[i][:max_chars]
  for j in range(len(t)):
    X_train[i, j, :] = word_vectors[t[j]]

for i in range(len(test_data_1)):
  t = test_data_1[i][:max_chars]
  for j in range(len(t)):
    X_test[i, j, :] = word_vectors[t[j]]

In [47]:
X_train.shape

(15131, 25, 128)

In [0]:
from keras.layers.recurrent import LSTM
#from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional, Dense, Activation, SimpleRNN, Dropout
from keras.models import Sequential

In [0]:
model = Sequential()
model.add(LSTM(units = 128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.10))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [0]:
batch_size = 32
nb_epochs = 10

In [0]:
# Total 20 epochs

In [52]:
model.fit(X_train, y_train_1,
          batch_size=batch_size,
          epochs=nb_epochs)

Epoch 1/10
15131/15131 [==============================] - 15s 998us/step - loss: 1.0721 - acc: 0.4103
Epoch 2/10
15131/15131 [==============================] - 14s 923us/step - loss: 1.0482 - acc: 0.4315
Epoch 3/10
15131/15131 [==============================] - 14s 921us/step - loss: 1.0388 - acc: 0.4478
Epoch 4/10
15131/15131 [==============================] - 14s 901us/step - loss: 1.0324 - acc: 0.4521
Epoch 5/10
15131/15131 [==============================] - 14s 922us/step - loss: 1.0282 - acc: 0.4555
Epoch 6/10
15131/15131 [==============================] - 14s 919us/step - loss: 1.0251 - acc: 0.4548
Epoch 7/10
15131/15131 [==============================] - 14s 915us/step - loss: 1.0220 - acc: 0.4565
Epoch 8/10
15131/15131 [==============================] - 14s 945us/step - loss: 1.0192 - acc: 0.4568
Epoch 9/10
15131/15131 [==============================] - 14s 918us/step - loss: 1.0210 - acc: 0.4612
Epoch 10/10
15131/15131 [==============================] - 14s 911us/step - loss: 

In [53]:
model.fit(X_train, y_train_1,
          batch_size=batch_size,
          epochs=nb_epochs)

Epoch 1/10
15131/15131 [==============================] - 14s 912us/step - loss: 1.0151 - acc: 0.4637
Epoch 2/10
15131/15131 [==============================] - 14s 902us/step - loss: 1.0136 - acc: 0.4634
Epoch 3/10
15131/15131 [==============================] - 14s 912us/step - loss: 1.0111 - acc: 0.4661
Epoch 4/10
15131/15131 [==============================] - 14s 900us/step - loss: 1.0205 - acc: 0.4630
Epoch 5/10
15131/15131 [==============================] - 14s 919us/step - loss: 1.0103 - acc: 0.4631
Epoch 6/10
15131/15131 [==============================] - 14s 912us/step - loss: 1.0102 - acc: 0.4676
Epoch 7/10
15131/15131 [==============================] - 14s 912us/step - loss: 1.0096 - acc: 0.4689
Epoch 8/10
15131/15131 [==============================] - 14s 915us/step - loss: 1.0047 - acc: 0.4687
Epoch 9/10
15131/15131 [==============================] - 14s 905us/step - loss: 1.0271 - acc: 0.4565
Epoch 10/10
15131/15131 [==============================] - 14s 944us/step - loss: 

In [0]:
y_pred = model.predict(X_test)

In [55]:
print(y_pred)

[[0.33288592 0.35646078 0.31065327]
 [0.33288592 0.35646078 0.31065327]
 [0.33396086 0.3571131  0.30892602]
 ...
 [0.3601683  0.37202573 0.2678059 ]
 [0.33288592 0.35646078 0.31065327]
 [0.33288592 0.35646078 0.31065327]]


In [0]:
y_pred_labels = []
for i in range(len(y_pred)):
  x = y_pred[i].argmax()
  if(x==0):
    y_pred_labels.append('negative')
  elif(x==1):
    y_pred_labels.append('neutral')
  else:
    y_pred_labels.append('positive')

In [0]:
y_actual_labels = test_labels.copy()

In [58]:
count = 0
for i in range(len(y_pred_labels)):
  if(y_pred_labels[i]==y_actual_labels[i]):
    count=count+1
Accuracy = count/len(y_actual_labels)
print(Accuracy)

0.4039593365436062


In [59]:
from sklearn.metrics import f1_score
F1 = f1_score(y_actual_labels, y_pred_labels, average=None)
print(F1)

[0.11538462 0.57189277 0.04      ]


In [60]:
from sklearn.metrics import recall_score
Recall = recall_score(y_actual_labels, y_pred_labels, average=None)
print(Recall)

[0.07317073 0.933687   0.02061856]


In [61]:
from sklearn.metrics import precision_score
Precision = precision_score(y_actual_labels, y_pred_labels, average=None)
print(Precision)

[0.27272727 0.41217799 0.66666667]
